<a href="https://colab.research.google.com/github/jonesLevin/LLMs/blob/main/Meeting_Minutes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create Meeting Minutes from an Audio File

In [1]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate torchaudio openai ollama colab-xterm httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 748.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [9]:
# imports

import os
import subprocess
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from google.colab import userdata
import ollama
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig, pipeline
import torch

In [3]:
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['OPENAI_TOKEN'] = userdata.get('OPENAI_TOKEN')

In [18]:
LLAMA = "meta-llama/Meta-Llama-3-8B-Instruct"
AUDIO_MODEL = "whisper-1"

In [6]:
audio_filename = '/content/denver_extract.mp3'
openai = OpenAI(api_key=os.environ['OPENAI_TOKEN'])

In [11]:
# Load the Whisper model from Hugging Face
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Transcribe the audio file, enabling timestamps for long audio
transcription = transcriber(audio_filename, return_timestamps=True)

# Print the transcribed text
print(transcription["text"])

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was us

 and kind of the confluence of this whole idea of the confluence week, the merging of two rivers. And as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let you see the creation of the logo here. And then, yes, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our Confluence Week is basically highlighting all of these Indigenous events and things that are happening around Denver so that we can and kind of bring more people together and kind of share this whole idea of indigenous people's day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the

In [12]:
system_prompt = """
You are an assistant that produces minutes from transcripts, with summary, key discussion points, takeaways and action
items with owners, in markdown.
"""
user_prompt = f"""
Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary
with attendees, location and date; discussion points: takeaways; and action items with owners {transcription["text"]}
"""

In [14]:
print(user_prompt)


Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary 
with attendees, location and date; discussion points: takeaways; and action items with owners.  and kind of the confluence of this whole idea of the confluence week, the merging of two rivers. And as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let you see the creation of the logo here. And then, yes, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our Confluence Week is basically highlighting all of these Indigenous events and things that are happening around Denver so that we can and kind of bring more people together and kind of share this whole idea of indigenous people's day. So thank you. Thank 

In [16]:
messages = [
    {'role': 'system', 'content': system_prompt},
    {'role': 'user', 'content': user_prompt}
]

In [17]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [19]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to('cuda')
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, quantization_config=bnb_config, device_map="auto")
outputs = model.generate(inputs, streamer=streamer, max_new_tokens=2000)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an assistant that produces minutes from transcripts, with summary, key discussion points, takeaways and action
items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary 
with attendees, location and date; discussion points: takeaways; and action items with owners.  and kind of the confluence of this whole idea of the confluence week, the merging of two rivers. And as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let you see the creation of the logo here. And then, yes, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more a

In [20]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [21]:
display(Markdown(response))

system

You are an assistant that produces minutes from transcripts, with summary, key discussion points, takeaways and action
items with owners, in markdown.user

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary 
with attendees, location and date; discussion points: takeaways; and action items with owners.  and kind of the confluence of this whole idea of the confluence week, the merging of two rivers. And as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let you see the creation of the logo here. And then, yes, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our Confluence Week is basically highlighting all of these Indigenous events and things that are happening around Denver so that we can and kind of bring more people together and kind of share this whole idea of indigenous people's day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the United States of America and to the republic for which it stands. All right, thank you councilman Lopez madam secretary roll call black Clark here Espanosa here Flynn Gilmore here Here cashman here can each here Lopez here new here or taiga Susman mr. President Here 11 present 11 members present. We do have a quorum approval of the minutes. Are there any corrections to the minutes of October 2nd? Seeing none minutes of October 2nd stand approve council announcements. Are there any announcements by members of council? Councilman Clark. Thank you, Mr. President. I just wanted to invite everyone down to the first ever Halloween parade on Broadway in Lucky District 7. It will happen on Saturday, October 21st at 6 o'clock p.m. It will move along Broadway from 3rd to Alameda. It's going to be a fun family friendly event. Everyone's invited to come down, wear a costume. There There'll be candy for the kids and there are Tiki zombies and 29 herses and all kinds of fun and funky stuff on the fun and funky part of Broadway. So please join us October 21st at six o'clock for the Broadway Halloween. Brad, thank you, Mr. President. All right, thank you, Councilman Clark. I will be there. All right, presentations. Madam Secretary, do we have any presentations? None, Mr. President. communications do we have any communications? None, Mr. President. We do have one proclamation this evening, proclamation 1127, an observance of the annual Indigenous Peoples Day in the City and County of Denver. Councilman Low players, will you please read it? Thank you, Mr. President. Pride. Proclamation number 17, well, let me just say this differently. Proclamation number 1127 series of 2017, and observance of the second annual Indigenous Peoples' Day in the City and County of Denver. Whereas the Council of the City and County of Denver recognizes that the Indigenous Peoples have lived and flourished on the lands known as the Amedica since time immemorial. And that Denver and the surrounding communities are built upon the ancestral homelands of numerous Indigenous tribes, which include the southern Yute Mountain, Yute tribes of Colorado, and whereas the tribal homelands and seasonal encampments of the Arapaho and Sham people along the banks of the Cherry Creek and South Platte River confluence gave bearing to the future settlements that would become the birthplace of the Mile High City. And whereas Colorado encompasses the ancestral homelands of 48 tribes in the city and county of Denver and surrounding communities are home to the descendants of approximately 100 tribal nations. And whereas, on October 3rd, 2016, the City and County of Denver unanimously passed Council Bill 801, series of 2016, officially designating the second Monday of October of each year, as Indigenous Peoples Day in Denver, Colorado. And whereas, the Council of the City and County of Denver continues to recognize and value the vast contributions contributions made to the community through indigenous peoples knowledge, science, philosophy, arts, and culture, and through these contributions the city of Denver has developed and thrived. Whereas the indigenous community, especially youth, have made great efforts this year to draw attention to the contributions of indigenous people including Confluence Week, drawing record We are detentives to a national indigenous youth leadership conference, leading conversations on inclusion with their peers, and supporting increased indigenous youth participation in science and engineering. Now therefore, be it proclaimed by the Council of the City and County of Denver, Section 1, that the Council of the City and County of Denver celebrates and honors the cultural and foundational contributions of indigenous people to our history. our past, present, and future. And continues to promote the education of the Denver community about these historical and contemporary contributions of indigenous people. Section two, at the City and County of Denver, Colorado does hereby observe October 9th, 2017 as indigenous people's day. Section three, at the clerk of the city and county of Denver shall attest and affix the seal of the city and county of Denver to this proclamation. And that a copy be transmitted, excuse me, to the Denver American Indian Commission, the city and county of Denver School District number one, and the Colorado Commission on Indian Affairs. Thank you, Councilman Lopez, your motion to adopt. President, I move that proclamation number 1127, series of 2017 be adopted. All right, it has been moved and seconded. Comes with members of council, Councilor Lopez. Thank you, Mr. President. It gives me a lot of pleasure and pride to read this proclamation officially for the third time, but as Indigenous Peoples Day in Denver officially for the second time. It is, it's always awesome to be able to see not just this proclamation come by my desk, but to see so many different people from our community in our council chambers. It was a very beautiful piece of artwork that you presented to us earlier. And it is exactly the spirit that we drafted this proclamation and this actual, the ordinance that created Indigenous Peoples Day when we sat down and wrote it. And as a community, we couldn't think of anything else to begin except for the confluence of the two rivers. And those confluence of the two rivers created such a great city. And we live in such an amazing city. And we're all proud of it. And sometimes we, and a lot of people from all over the country or all over the world are proud of it. And sometimes a little too proud of it, is just telling them to go back home. But I'm kidding when I say that. But the really nice thing about this is that we are celebrating Indigenous people's day out of pride for who we are, who we are as a city, and the contributions of Indigenous people to the city, not out of spite, not out of a replacement of one culture over the other, or out of contempt, or disrespect. You know, I think of a quote that Cesar Chavez made very popular, and it stuck with me for a very long time. And anytime I have the opportunity, I speak in front of children, and especially children in our community that, often second guess themselves on where they're coming from, who they are. And I always say that it's very important to be proud of who you're from. And the quote that I use from Cesar Chavez is, pride in one's own culture does not require contempt or disrespect of another, right? And that's very important. It's very important for us to recognize that. No matter who we are, where we come from in this society, that your pride in your own culture doesn't require, should not require the contempt or disrespect of another. And man, what a year to be, for that to just sit on our shoulders for a while, for us to think about, right? And so I wanted to just to thank you all, I think the commission, there's gonna be a couple individuals that are gonna come speak, thank you for your art, your lovely artwork for us to see what's in your heart and what now has become, probably is gonna be a very important symbol for the community. And also just for the work, the daily work every single day, we still have a lot of brothers and sisters whose ancestors once lived in these lands freely now stand on street corners, right, in poverty. without access to services, right? Without access to sobriety or even housing or jobs. And what a cruel way to pay back a culture that has paved the way for the city to be built upon its shores, right? So we have a lot of work to do. And these kind of proclamations in this day is not a day off, it's a day on in Denver, right? right and addressing those critical issues. So I know that my colleagues are very supportive. I'm gonna ask you to support this proclamation as I know you always have done in the past. I'm very proud of today. Oh, and we made Time Magazine and Newsweek once again today as being a leader in terms of the cities that are celebrating Indigenous Peoples Day. I wanted to make a point out of that. Thank you, Councilman Lopez, and thank you for sponsoring this. Council Martega. Mr. President, I want to ask that my name be added. I don't think I could add much more to what Councilman Lopez has shared with us. I want to thank him for bringing this forward and really just appreciate all the contributions that our Native American community has contributed to this great city and great state. I worked in the Lieutenant Governor's Office when the commission on Indian affairs was created and had the benefit of being able to go down to the four corners for a peace treaty signing ceremony between the youths and the Comanches that had been sort of at odds with each other for about 100 years. And just being able to participate in that powwow was pretty awesome. So and for those of you who continue to participate in the annual powwow, It's such a great opportunity for everybody else to enjoy so many of the contributions of the culture. I mean, to see that the dance continues to be carried on, as well as the native language, from generation to generation is just so incredible, because in so many cultures, people have come here and assimilated to the norms here, and they lose their language. and lose a lot of the culture. And in the native community, that hasn't happened. That has, you know, commitment to just passing that on from generation to generation is so important. And so I'm happy to be a co-sponsor of this tonight. Thank you. All right, thank you, Councilwoman Artega. Councilwoman Keneige. Thank you very much. And I also want to thank my colleague for bringing this forward. And I just wanted to say a word to the artist about how beautiful and moving I thought this logo was and your description of it. And I think one of the things that is clear is, the words sometimes don't convey the power of imagery or music or the other pieces that make up culture. And so I think the art is so important. And when you talked about water, I was also thinking about land. And I guess I just wanted to say thank you, Many of the Native American peoples of Colorado have been at the forefront, or actually nationally, of defending some of the public lands that have been protected over the last few years that are under attack right now. And there are places that the communities have fought to protect, but that everyone gets to enjoy. And so I just think that it's an example of where cultural preservation intersects with environmental protection, with recreation, and all of the other ways that public lands so important and so I think I just wanted to say thank you for that because I think we have some very sacred places in our country that are at risk right now and So as we celebrate I appreciate that there's still a piece of resistance in here And I think that I just want to mention a solidarity and I mentioned a feeling of solidarity with that resistance So thank you and Happy Confluence Week Thank you councilman kinech and See no other comments. I'll just say a couple and In a time of such divisive Ugliness and just despicable behavior from our leadership The reason I'm so supportive of indigenous people's days because it means inclusivity it means respecting all respecting those who have been silenced On purpose for a long time and whose history has not been told and so we celebrate inclusivity in the face of such evil times, honestly.assistant

**Minutes of the Denver City Council Meeting**

**Date:** Monday, October 9th
**Location:** City Council Chambers
**Attendees:** 11 members of the City Council, City officials, and guests

**Summary:** The meeting began with the Pledge of Allegiance, followed by the approval of the minutes from October 2nd. Councilman Clark announced the first-ever Halloween parade on Broadway in District 7. The council then considered Proclamation 1127, an observance of the annual Indigenous Peoples Day in the City and County of Denver.

**Discussion Points:**

* The importance of recognizing and honoring the cultural and foundational contributions of indigenous people to the city's history, present, and future.
* The significance of the confluence of the two rivers and its connection to the city's founding.
* The need to promote education and awareness about the historical and contemporary contributions of indigenous people.
* The importance of inclusivity and respect for all cultures, particularly in a time of divisive behavior and despicable leadership.

**Takeaways:**

* The city's celebration of Indigenous Peoples Day is not just a day off, but a day on, recognizing the contributions of indigenous people to the city's development.
* The city's recognition of Indigenous Peoples Day is a way to promote education and awareness about the historical and contemporary contributions of indigenous people.
* The confluence of the two rivers is a symbol of the city's founding and the cultural and foundational contributions of indigenous people.

**Action Items:**

1. **Proclamation 1127:** The council adopted Proclamation 1127, an observance of the annual Indigenous Peoples Day in the City and County of Denver.
2. **Confluence Week:** The council recognized the importance of Confluence Week, which highlights indigenous events and activities in Denver, and encouraged community members to participate.
3. **Promoting Education and Awareness:** The council encouraged the promotion of education and awareness about the historical and contemporary contributions of indigenous people, particularly among youth and in schools.

**Owners:**

* Councilman Lopez: Sponsor of Proclamation 1127
* Councilwoman Artega: Co-sponsor of Proclamation 1127
* Councilman Kinech: Co-sponsor of Proclamation 1127
* Artist: Creator of the logo and artwork presented during the meeting